In [424]:
import numpy as np
import pandas as pd
import bs4
import folium
import requests
# import geopy
import urllib3

Grabbing URL with urllib and scraping the html with Beautiful Soup

In [425]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
urlLib = urllib3.PoolManager()
r = urlLib.request("GET", url)

soup = bs4.BeautifulSoup(r.data)

C:\Users\Schmuck\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Running through the first table and turning it into a pandas dataframe

In [426]:
table = soup.findChildren("table")[0]
rows = []
for row in table.find_all("tr"):
        temp = []
        elems = row.find_all("td")
        for elem in elems:
            if elems[-1] == elem:
                text = elem.text
                text = text[:-1]
                temp.append(text)
            else:
                temp.append(elem.text)
        rows.append(temp)
    
df = pd.DataFrame.from_records(rows[1:])
headers = [head.text for head in table.find_all("th")]
headers[-1] = headers[-1][:-1]
df.columns = headers

print(df.shape)
df.head()

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping all the "Not assigned" boroughs and assign the borough names for neighbourhoods that lack a name

In [427]:
mask = df["Borough"] != "Not assigned"
df = df[mask]
print(df.shape)
df.head()

row = df[df["Neighbourhood"] == "Not assigned"]
df.at[row.index, "Neighbourhood"] = row.Borough.values[0]

(211, 3)


Gather all the unique postal code values

In [428]:
code_list = df["Postcode"].unique()
print(code_list)

['M3A' 'M4A' 'M5A' 'M6A' 'M7A' 'M9A' 'M1B' 'M3B' 'M4B' 'M5B' 'M6B' 'M9B'
 'M1C' 'M3C' 'M4C' 'M5C' 'M6C' 'M9C' 'M1E' 'M4E' 'M5E' 'M6E' 'M1G' 'M4G'
 'M5G' 'M6G' 'M1H' 'M2H' 'M3H' 'M4H' 'M5H' 'M6H' 'M1J' 'M2J' 'M3J' 'M4J'
 'M5J' 'M6J' 'M1K' 'M2K' 'M3K' 'M4K' 'M5K' 'M6K' 'M1L' 'M2L' 'M3L' 'M4L'
 'M5L' 'M6L' 'M9L' 'M1M' 'M2M' 'M3M' 'M4M' 'M5M' 'M6M' 'M9M' 'M1N' 'M2N'
 'M3N' 'M4N' 'M5N' 'M6N' 'M9N' 'M1P' 'M2P' 'M4P' 'M5P' 'M6P' 'M9P' 'M1R'
 'M2R' 'M4R' 'M5R' 'M6R' 'M7R' 'M9R' 'M1S' 'M4S' 'M5S' 'M6S' 'M1T' 'M4T'
 'M5T' 'M1V' 'M4V' 'M5V' 'M8V' 'M9V' 'M1W' 'M4W' 'M5W' 'M8W' 'M9W' 'M1X'
 'M4X' 'M5X' 'M8X' 'M4Y' 'M7Y' 'M8Y' 'M8Z']


Drop all duplicates and assign new neighbourhood column with the join neighbourhood data 

In [429]:
df_dropped = df.drop_duplicates(["Postcode"], keep = "first")
df_dropped.drop("Neighbourhood", axis = 1, inplace = True)
print("Number of Unique Postal Codes: {}".format(df_dropped.shape[0]))

neighbourhoods = []

for code in code_list:
    hoods = df["Neighbourhood"][df["Postcode"] == code].values
    neighbourhoods.append(",".join(hoods))
df_dropped["Neighbourhood"] = neighbourhoods
df_dropped.reset_index(drop = True, inplace = True)
df_dropped.head()

Number of Unique Postal Codes: 103


C:\Users\Schmuck\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Printing shape of the dataset

In [430]:
df = df_dropped
print(df.shape)

# # df.drop(["Latitude", "Longitude"], axis = 1, inplace = True)
# df.head()

(103, 3)


In [431]:
# Tried to run geocoder, loop ran for 30 seconds with no output on a single postal code. Decided to simply use the csv provided

# import geocoder

# coords = None

# while coords is None:
#     g = geocoder.google("M2K, Toronto, Ontario")
#     coords = g.latlng
# print(coords)

Using join method to create a new dataframe of the existing data along with the latitude and longitude data.

In [432]:
latLon = pd.read_csv("Geospatial_Coordinates.csv")
latLon.head()

latLon.sort_values(by = "Postal Code", inplace = True)
df.sort_values(by = "Postcode", inplace = True)

df = df.join(latLon.set_index("Postal Code"), on = "Postcode")
df.reset_index(drop = True, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Practice requesting data from the Foursquare API using the first location in DataFrame

In [433]:
import requests
import json

ll = df.iloc[0][["Latitude", "Longitude"]].values
value = ",".join(str(val) for val in ll)

url = "https://api.foursquare.com/v2/venues/explore"
params = dict(
    client_id = 'W5CHCMS4RL2BMAQVYOORLJQCF4XLGWK42SDQWAG1XFLX2LRV',
    client_secret = 'BVUEP1F3ZHWSQGN24XBSDK0GK15CBU2OJZ53VLQIR4PQQQ4U',
    v = '20180323',
    ll = value,
    limit = 10,
    radius = 600)

# print(params)
req = requests.get(url = url, params = params)
data = json.loads(req.text)

Showing all data in response section of json file with their respective keys

In [434]:
keys = data["response"].keys()

for key in keys:
    print("--" + key + "--")
    print(data["response"][key], "\n")

--headerLocation--
Malvern 

--headerFullLocation--
Malvern, Toronto 

--headerLocationGranularity--
neighborhood 

--totalResults--
4 

--suggestedBounds--
{'ne': {'lat': 43.8120863054, 'lng': -79.1868848168765}, 'sw': {'lat': 43.80128629459999, 'lng': -79.20182198312352}} 

--groups--
[{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4d669cba83865481c948fa53', 'name': 'Images Salon & Spa', 'contact': {}, 'location': {'address': '8130 Sheppard Ave E', 'crossStreet': 'Morningside Ave', 'lat': 43.80228301948931, 'lng': -79.19856472801668, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80228301948931, 'lng': -79.19856472801668}], 'distance': 595, 'postalCode': 'M1B 3W3', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)', 'Toronto ON M1B 3W3', 'C



The section "group" seems to have the majority of the valuable information contained in it. 

The total results section would be an interesting thing to explore as it can be an indicator of density in the given radius.

Further investigating the "groups" section to understand json file structure

In [435]:

results = data["response"]["groups"][0]["items"]
venue_keys = list(results[0]['venue'].keys())
for key in venue_keys:
    print("--{}--".format(key))
    print(results[0]["venue"][key], "\n")

--id--
4d669cba83865481c948fa53 

--name--
Images Salon & Spa 

--contact--
{} 

--location--
{'address': '8130 Sheppard Ave E', 'crossStreet': 'Morningside Ave', 'lat': 43.80228301948931, 'lng': -79.19856472801668, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80228301948931, 'lng': -79.19856472801668}], 'distance': 595, 'postalCode': 'M1B 3W3', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)', 'Toronto ON M1B 3W3', 'Canada']} 

--categories--
[{'id': '4bf58dd8d48988d1ed941735', 'name': 'Spa', 'pluralName': 'Spas', 'shortName': 'Spa', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_', 'suffix': '.png'}, 'primary': True}] 

--verified--
False 

--stats--
{'tipCount': 0, 'usersCount': 0, 'checkinsCount': 0, 'visitsCount': 0} 

--beenHere--
{'count': 0, 'lastCheckinExpiredAt': 0, 'marked': False, 'unconfirmedCount': 0} 

--photos--
{'count': 0, 'groups': []} 

--hereNow--
{'count': 0, 

Simlar to the New York city example. Grabbing the types of places in the area will be a good feature to draw on. Another feature could be the amount of check ins or visits that a venue has. Could be an indicator of community engagment. Let's check out some other neighborhoods and see how our results differ from this one. There were only 4 results returned here which seems like a small amount.

First lets make it easier to grab and format the lat and long values from our DataFrame with a quick function and a function to make requests easier

In [676]:
### Takes in a row number, i,  and returns the corresponding lat and long values
def get_lat_long(i):
    ll = df.iloc[i][["Latitude", "Longitude"]].values
    return ",".join(str(val) for val in ll)

### Lets also make a quick way to request from the Foursquare API
#Input : value - row number we want to learn about
#        rad (optional: default = .5 mile) - radius around search point
#        limit (optional: default = 15) - number of results to output from the API
#Output:
#       num_results - number of results from request
#       data - "important" data from the groups section of the json file

def makeRequest(value, rad = 805, limit = 15):
    ll = get_lat_long(value)
    url = "https://api.foursquare.com/v2/venues/explore"
    params = dict(
        client_id = 'W5CHCMS4RL2BMAQVYOORLJQCF4XLGWK42SDQWAG1XFLX2LRV',
        client_secret = 'BVUEP1F3ZHWSQGN24XBSDK0GK15CBU2OJZ53VLQIR4PQQQ4U',
        v = '20180323',
        ll = ll,
        limit = limit,
        radius = rad)
    req = requests.get(url = url, params = params)
    data = json.loads(req.text)["response"]
    try:
        total = data["totalResults"]
    except:
        total = 0
    return total, data["groups"]


Lets test it out on a couple more rows

In [677]:
# Random numbers
test_rows = [10, 36, 58, 86, 99, 101]
for row in test_rows:
    n, new_data = makeRequest(row)
    print("Number of Results from Request: {}".format(n))

Number of Results from Request: 17
Number of Results from Request: 13
Number of Results from Request: 236
Number of Results from Request: 50
Number of Results from Request: 11
Number of Results from Request: 13


There seems to be a good variety of result sizes. Now lets check out all the neighborhood location on a map

In [763]:
toronto_lat_long = [43.7, -79.4]
m = folium.Map(location = toronto_lat_long, zoom_start = 11, tiles = "Stamen Toner")

for row in range(df.shape[0]):
    folium.Circle(location = df.iloc[row][["Latitude", "Longitude"]].values,
                  color = "crimson", 
                  popup = df.iloc[row]["Neighbourhood"],
                  fill = True,
                  radius = 90).add_to(m)
m

Lets grab all the information that we are interested in and put it in a single DataFrame to work with moving forward.

In [715]:
records = []

for row in range(df.shape[0]):
#     print(row)
    n, data = makeRequest(row)
    data = data[0]["items"]
    for venue in data:
        records.append([
            df.iloc[row]["Neighbourhood"],
            venue["venue"]["name"],
            venue["venue"]["categories"][0]["name"],
            venue["venue"]["beenHere"]["count"], #Need to see if any values besides 0 show up in data
            venue["venue"]["location"]["distance"],
            n,
        ])

We will turn it into a DataFrame and make give it some column names

In [720]:
data = pd.DataFrame.from_records(records)
data.columns = ["Neighbourhood", "Venue Name", "Venue Category", "Been Here", "Distance from Query", "Total Query Results"]
print(data.shape)
data.head()

(1305, 6)


,Neighbourhood,Venue Name,Venue Category,Been Here,Distance from Query,Total Query Results
0,"Rouge,Malvern",Images Salon & Spa,Spa,0,595,12
1,"Rouge,Malvern",Wendy's,Fast Food Restaurant,0,387,12
2,"Rouge,Malvern",Wendy's,Fast Food Restaurant,0,600,12
3,"Rouge,Malvern",Staples Morningside,Paper / Office Supplies Store,0,735,12
4,"Rouge,Malvern",Harvey's,Fast Food Restaurant,0,796,12


It looks like all the been here data is 0. Not sure if that is an API issue or if I am pulling from the wrong area, but we will just have to throw that column out!

In [721]:
data.drop("Been Here", axis = 1, inplace = True)
data.head()

,Neighbourhood,Venue Name,Venue Category,Distance from Query,Total Query Results
0,"Rouge,Malvern",Images Salon & Spa,Spa,595,12
1,"Rouge,Malvern",Wendy's,Fast Food Restaurant,387,12
2,"Rouge,Malvern",Wendy's,Fast Food Restaurant,600,12
3,"Rouge,Malvern",Staples Morningside,Paper / Office Supplies Store,735,12
4,"Rouge,Malvern",Harvey's,Fast Food Restaurant,796,12


Lets turn our distance from query into a boolean column that indicates if the location is within a quarter mile of the location that we searched. This should be an indicator of the density of the neighborhood. Could add more columns with smaller and/or larger distances, but I think this should be fine for now.

In [722]:
data["Within .25 mile"] = pd.get_dummies(data["Distance from Query"] < 403)[True]
data.drop("Distance from Query", axis = 1, inplace = True)
print(data.shape)
data.head()

(1305, 5)


,Neighbourhood,Venue Name,Venue Category,Total Query Results,Within .25 mile
0,"Rouge,Malvern",Images Salon & Spa,Spa,12,0
1,"Rouge,Malvern",Wendy's,Fast Food Restaurant,12,1
2,"Rouge,Malvern",Wendy's,Fast Food Restaurant,12,0
3,"Rouge,Malvern",Staples Morningside,Paper / Office Supplies Store,12,0
4,"Rouge,Malvern",Harvey's,Fast Food Restaurant,12,0


Determining the percentge of "Unique" venue in the area. Could be an indicator of how "local" vs "corporate" the establishments in the area are.

In [723]:
uniquePercent = data.groupby("Neighbourhood")["Venue Name"].nunique()/data.groupby("Neighbourhood")["Venue Name"].count()
uniquePercent.rename("Unique Percent", inplace = True)
uniquePercent.head()

Neighbourhood
Adelaide,King,Richmond                                                                                    1.000000
Agincourt                                                                                                 1.000000
Agincourt North,L'Amoreaux East,Milliken,Steeles East                                                     1.000000
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown    0.923077
Alderwood,Long Branch                                                                                     1.000000
Name: Unique Percent, dtype: float64

Lets find the average value of the types of venues in the area and adding the other categories to our final dataset

In [724]:
venue_cat = pd.get_dummies(data["Venue Category"])
venue_cat["Neighbourhood"] = data["Neighbourhood"]
columns = venue_cat.columns.tolist()
columns = columns[-1:] + columns[:-1]
venue_cat = venue_cat[columns]
venue_cat = venue_cat.groupby("Neighbourhood").mean()
total_num = data[["Neighbourhood", "Total Query Results"]].drop_duplicates().set_index("Neighbourhood")
prox_data = data.groupby("Neighbourhood")["Within .25 mile"].sum()
appending = pd.DataFrame([uniquePercent, prox_data]).transpose()
appending = appending.join(total_num)
venue_cat = venue_cat.join(appending)
print(venue_cat.shape)
venue_cat.tail()

(102, 228)


,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Unique Percent,Within .25 mile,Total Query Results
Neighbourhood,,,,,,,,,,,,,,,,,,,,,
Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,5.0,8
Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.0,6
"Woodbine Gardens,Parkview Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933333,9.0,17
Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,4.0,13
York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,3.0,7


Finally, lets run our KMeans algorithm on the dataset and find out what it thinks

In [756]:
from sklearn.cluster import KMeans

k = 6

venue_cat.reset_index(drop = True, inplace = True)
model = KMeans(n_clusters = k, random_state = 1).fit(venue_cat)

output = model.labels_

There was a mismatch in the data so I dropped the missing rows added the labels from KMeans to the final dataset

In [757]:
from collections import Counter

final = df[["Neighbourhood", "Latitude", "Longitude"]].set_index("Neighbourhood")
last_neigh = list(data["Neighbourhood"].unique())
first_neigh = df["Neighbourhood"].values


missing = Counter(first_neigh) - Counter(last_neigh)
missing = list(missing.keys())[0]
final.drop(missing, inplace = True)
final["Labels"] = output
final.head()

,Latitude,Longitude,Labels
Neighbourhood,,,
"Rouge,Malvern",43.806686,-79.194353,1
"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0
"Guildwood,Morningside,West Hill",43.763573,-79.188711,0
Woburn,43.770992,-79.216917,0
Cedarbrae,43.773136,-79.239476,0


Lets visualize those labels on the map now

In [762]:
m = folium.Map(location = toronto_lat_long, zoom_start = 11, tiles = "Stamen Toner")

for row in range(final.shape[0]):
    current = final.iloc[row]
    label = current["Labels"]
    if label == 0:
        label_color = "crimson"
    elif label == 1:
        label_color = "blue"
    elif label == 2:
        label_color = "orange"
    elif label == 3:
        label_color = "green"
    elif label == 4:
        label_color = "purple"
    elif label == 5:
        label_color = "yellow"
    folium.Circle(location = current[["Latitude", "Longitude"]].values,
                  popup = current.name,
                  color = label_color,
                  fill = True,
                  radius = 90).add_to(m)

m